In [34]:
import pandas as pd
import numpy as np
import re

In [35]:
path='C:\\Users\\camendol\\Downloads\\'
file='II 7.2 BPO Input Requirements.xlsx'

file_path=path+file

In [36]:
xl_file = pd.ExcelFile(file_path)

xl_file.sheet_names  

['General Questions', 'Table Input', 'Field Input']

In [37]:
column_data=xl_file.parse('Field Input')

In [38]:
column_data.head()

,Input Table Type,Table,Field,Description (from OPA Spec),Type,Default,Nulls?,PK?,Foreign Key,Notes,Spec Question,Reviewed
0,Required,MEMBER,MEMBER,Unique code issued by a healthplan to identify...,varchar (32),NaN,No,Y,NaN,GRP_MPI --> Anna's Dedupe Stuff,0,1
1,Required,MEMBER,SEX,Member's sex code.,bit,NaN,No,NaN,NaN,Valid Values:\n0=Male \n1=Female\nHow to handl...,0,1
2,Required,MEMBER,DOB,Member's date of birth. Formatted as YYYYMMDD.,datetime,NaN,No,NaN,NaN,"If null, we should exclude the member. This is...",0,1
3,Required,MEMBER,STATE_N,Member's state code,char (2),NaN,Yes,NaN,NaN,SPEC: Is this information allowed to vary acro...,0,1
4,Required,MEMBER,ZIP_N,Member's zip code,varchar (10),NaN,Yes,NaN,NaN,Can be a mix of 5 or 9 -- Don't need to strip ...,0,1


In [39]:
column_data['Table'].unique()

array(['MEMBER', 'PROVIDER', 'SERVICEMED', 'SUBSCRIBER',
       'CUSTOM_PROVIDER_STATUS_SPAN', 'LABDATA', 'PREMIUM', 'SERVICERX',
       'CUSTOM_ACCT', 'CUSTOM_AT_RISK_STATUS', 'CUSTOM_BENEFIT_PLAN',
       'CUSTOM_BIZ_SEGMENT', 'CUSTOM_CONTRACT', 'CUSTOM_CONTRACT_TYPE',
       'CUSTOM_DENIED_IND', 'CUSTOM_EMPLOYEE_TYPE',
       'CUSTOM_MEM_USERDEF_1', 'CUSTOM_MEM_USERDEF_2',
       'CUSTOM_MEM_USERDEF_3', 'CUSTOM_MEM_USERDEF_4',
       'CUSTOM_NETWORK_PAID_STATUS', 'CUSTOM_PROD',
       'CUSTOM_PROV_USERDEF_2', 'CUSTOM_PROVAFFIL',
       'CUSTOM_PROVIDER_STATUS', 'CUSTOM_RISK_TYPE', 'CUSTOM_SPEC_RX_N',
       'CUSTOM_COVERAGE_STATUS'], dtype=object)

In [40]:
column_data['Type'].unique()

array(['varchar (32)', 'bit', 'datetime', 'char (2)', 'varchar (10)',
       'varchar (30)', 'smalldatetime', 'varchar (20)', 'varchar (6)',
       'varchar (50)', 'varchar (100)', 'varchar (15)', 'decimal (9, 6)',
       'varchar (8)', 'varchar (7)', 'varchar (4)', 'tinyint',
       'varchar (3)', 'varchar (2)', 'int', 'decimal (19, 2)', 'char (1)',
       'bigint - Identity', 'varchar (25)', 'smallint', 'varchar (11)',
       'decimal(4,2)', 'varchar (150)'], dtype=object)

In [41]:
informat_dict={ 'varchar (32)':'$32.'
               ,'bit':'$1.'
               ,'datetime':'yymmdd10.'
               ,'char (2)':'$2.'
               ,'varchar (10)':'$10.'
               ,'varchar (30)':'$30.'
               ,'smalldatetime':'8'
               ,'varchar (20)':'$20.'
               ,'varchar (6)':'$6.'
               ,'varchar (50)':'$50.'
               ,'varchar (100)':'$100.'
               ,'varchar (15)':'$15.'
               ,'decimal (9, 6)':'8.'
               ,'varchar (8)':'$8.'
               ,'varchar (7)':'$7.'
               ,'varchar (4)':'$4.'
               ,'tinyint':''
               ,'varchar (3)':'$3.'
               ,'varchar (2)':'$2.'
               ,'int':'8'
               ,'decimal (19, 2)':''
               ,'char (1)':'$1.'
               ,'bigint - Identity':''
               ,'varchar (25)':'$25.'
               ,'smallint':'8.'
               ,'varchar (11)':'$11.'
               ,'decimal(4,2)':''
               ,'varchar (150)':'$150.' }

In [54]:
length_dict={ 'varchar (32)':'32'
             ,'bit':'1'
             ,'datetime':'8'
             ,'char (2)':'2'
             ,'varchar (10)':'10'
             ,'varchar (30)':'30'
             ,'smalldatetime':'8'
             ,'varchar (20)':'20'
             ,'varchar (6)':'6'
             ,'varchar (50)':'50'
             ,'varchar (100)':'100'
             ,'varchar (15)':'15'
             ,'decimal (9, 6)':'8'
             ,'varchar (8)':'8'
             ,'varchar (7)':'7'
             ,'varchar (4)':'4'
             ,'tinyint':''
             ,'varchar (3)':'3'
             ,'varchar (2)':'2'
             ,'int':'8'
             ,'decimal (19, 2)':'8'
             ,'char (1)':'1'
             ,'bigint - Identity':'8'
             ,'varchar (25)':'25'
             ,'smallint':'8'
             ,'varchar (11)':'11'
             ,'decimal(4,2)':'8'
             ,'varchar (150)':'150' }

In [55]:
format_dict={ 'varchar (32)':' '
             ,'bit':' '
             ,'datetime':'yymmdd10.'
             ,'char (2)':' '
             ,'varchar (10)':' '
             ,'varchar (30)':' '
             ,'smalldatetime':' '
             ,'varchar (20)':' '
             ,'varchar (6)':' '
             ,'varchar (50)':' '
             ,'varchar (100)':' '
             ,'varchar (15)':' '
             ,'decimal (9, 6)':' '
             ,'varchar (8)':' '
             ,'varchar (7)':' '
             ,'varchar (4)':' '
             ,'tinyint':' '
             ,'varchar (3)':' '
             ,'varchar (2)':' '
             ,'int':' '
             ,'decimal (19, 2)':' '
             ,'char (1)':' '
             ,'bigint - Identity':' '
             ,'varchar (25)':' '
             ,'smallint':' '
             ,'varchar (11)':' '
             ,'decimal(4,2)':' '
             ,'varchar (150)':' ' }

In [72]:
#name                  |informat |length|format   |label|transform    ||
#MEMBERID               |$32.     |32    |         |'---'|/*Transform*/|
column_data['name']=column_data['Field']
column_data['format']=column_data['Type'].replace(format_dict)
column_data['length']=column_data['Type'].replace(length_dict)
column_data['informat']=column_data['Type'].replace(informat_dict)
column_data['label']=column_data['Description (from OPA Spec)'].replace('\\n\\r',' ',regex=True)

In [73]:
column_data.loc[:,('name','informat','length','format','label','transform')]

,name,informat,length,format,label,transform
0,MEMBER,$32.,32,,Unique code issued by a healthplan to identify...,/*Transform*/
1,SEX,$1.,1,,Member's sex code.,/*Transform*/
2,DOB,yymmdd10.,8,yymmdd10.,Member's date of birth. Formatted as YYYYMMDD.,/*Transform*/
3,STATE_N,$2.,2,,Member's state code,/*Transform*/
4,ZIP_N,$10.,10,,Member's zip code,/*Transform*/
5,COVERAGE_STATUS,$30.,30,,Member's relationship to subscriber.,/*Transform*/
6,EFF_DT,8,8,,Enrollment effective date of coverage. Formatt...,/*Transform*/
7,END_DT,8,8,,Enrollment end date of coverage. Formatted as ...,/*Transform*/
8,RX,$1.,1,,Code which indicates whether the member's phar...,/*Transform*/
9,MED,$1.,1,,Code which indicates whether the member's medi...,/*Transform*/


In [74]:
outfile=path+'test.schema'
column_data.to_csv(outfile, sep='|')